# NCAA Men's Basketball Pairwise Rankings
Generate rankings for college basketball by pairwise comparisons

In [1]:
import numpy as np
import pandas as pd

## Import Data

In [2]:
teams = pd.read_csv('data/2019/Teams.csv')
results = pd.read_csv('data/2019/RegularSeasonCompactResults.csv')

In [3]:
def clean_data(teams, results, season):
    active_teams = teams.set_index('TeamID')[['TeamName']]
    season_results = results[results['Season'] == season]
    for team_id in list(teams.TeamID):
        wins = season_results[season_results['WTeamID'] == team_id]
        losses = season_results[season_results['LTeamID'] == team_id]
        if len(wins) + len(losses) > 0:
            active_teams.at[team_id, 'Wins'] = len(wins)
            active_teams.at[team_id, 'Losses'] = len(losses)
        else:
            active_teams.drop(team_id, inplace=True)
    return (active_teams, season_results)

In [4]:
active_teams, season_results = clean_data(teams, results, 2018)

## RPI - Ratings Percentage Index

* The team's own winning percentage (25%)

In [5]:
def WP(team_id, active_teams, season_results):
    wins = season_results[season_results['WTeamID'] == team_id]
    losses = season_results[season_results['LTeamID'] == team_id]
        
    weighted_wins, weighted_losses = 0, 0
    for game_id in wins.index:
        if wins.at[game_id, 'WLoc'] == 'A': weighted_wins += 1.3
        elif wins.at[game_id, 'WLoc'] == 'H': weighted_wins += 0.7
        else: weighted_wins += 1
    for game_id in losses.index:
        if losses.at[game_id, 'WLoc'] == 'A': weighted_losses += 1.3
        elif losses.at[game_id, 'WLoc'] == 'H': weighted_losses += 0.7
        else: weighted_losses += 1
    
    return weighted_wins/(weighted_wins+weighted_losses)

In [6]:
for team_id in active_teams.index:
    active_teams.at[team_id, 'WP'] = WP(team_id, active_teams, season_results)

* The average of the team's opponents' winning percentages (50%)

In [7]:
def OWP(team_id, active_teams, season_results):
    wins = list(season_results[season_results['WTeamID'] == team_id].LTeamID)
    losses = list(season_results[season_results['LTeamID'] == team_id].WTeamID)
    
    accumulator = 0
    for opp_id in wins+losses:
        # wins and losses excluding games against the TUC
        opp_wins = season_results[(season_results['WTeamID'] == opp_id) & (season_results['LTeamID'] != team_id)]
        opp_losses = season_results[(season_results['LTeamID'] == opp_id) & (season_results['WTeamID'] != team_id)]
        accumulator += len(opp_wins)/(len(opp_wins)+len(opp_losses))
    return accumulator/(len(wins)+len(losses))

In [8]:
for team_id in active_teams.index:
    active_teams.at[team_id, 'OWP'] = OWP(team_id, active_teams, season_results)

* The average of the team's opponents opponents' winning percentages (25%)

In [9]:
def OOWP(team_id, active_teams, season_results):
    wins = list(season_results[season_results['WTeamID'] == team_id].LTeamID)
    losses = list(season_results[season_results['LTeamID'] == team_id].WTeamID)
    
    accumulator = 0
    for opp_id in wins+losses:
        accumulator += active_teams.at[opp_id, 'OWP']
    return accumulator/(len(wins)+len(losses))

In [10]:
for team_id in active_teams.index:
    active_teams.at[team_id, 'OOWP'] = OOWP(team_id, active_teams, season_results)

* Combined RPI Formula

In [11]:
for team_id in active_teams.index:
    active_teams.at[team_id, 'RPI'] = (active_teams.at[team_id, 'WP']*0.25) + (active_teams.at[team_id, 'OWP']*0.5) + (active_teams.at[team_id, 'OOWP']*0.25)
active_teams.sort_values(by='RPI', ascending=False).head()

,TeamName,Wins,Losses,WP,OWP,OOWP,RPI
TeamID,,,,,,,
1438,Virginia,31.0,2.0,0.935897,0.607327,0.563329,0.678470
1437,Villanova,30.0,4.0,0.895385,0.603407,0.558442,0.665160
1462,Xavier,28.0,5.0,0.844884,0.600805,0.562336,0.652207
1242,Kansas,27.0,7.0,0.780564,0.626923,0.559004,0.648354
1314,North Carolina,25.0,10.0,0.711656,0.650478,0.565324,0.644484


## Record vs. Common Opponents

## Head-to-Head Record

## Pairwise Comparison